# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [212]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.ml import feature
from pyspark.ml import regression
from pyspark.sql import functions as fn
from pyspark.sql import Row
from pyspark import sql

## Warning: Use exclusively Spark. Do not use Pandas at all in this assignment

# Part 2: Dataframes and Spark ML

In this section, you will learn to create dataframes from messy data and then perform simple regression on it.

There is some mysterious process generating data, stored in `/datasets/host_server_requests`, with the following format:

`feature1|feature2|...|featurem => outcome`

`feature1` can be either "HOST" or "SERVER" and from feature $2$ through $m$ are floating point numbers.

In [3]:
requests_rdd = sc.textFile('/datasets/host_server_requests')

## Question 1:

In this question, you will create a function `process_line` that receives a line from `/datasets/host_server_requests` and returns a `Row` object with the following columns: 

- You will codify the first feature as a column `f1` with a `1` if the source is `HOST` and `0` otherwise
- You will create 7 other features that you assign to columns `f2`, `f3`, ..., through `f8`
- Finally, you will assign the outcome to the column `label`
- Remember to make all features of type `float`.

For the following code:


```python
requests_rdd.map(process_line).take(10)
```

it should generate the following:

```python
[Row(f1=1.0, f2=2e-05, f3=0.80279, f4=-0.09174, f5=0.04041, f6=-0.22504, f7=-0.0504, f8=0.58149, label=163.877101489),
 Row(f1=1.0, f2=5e-05, f3=-0.00454, f4=-0.0211, f5=0.00174, f6=-0.11684, f7=0.19182, f8=-0.23745, label=-105.023368852),
 Row(f1=1.0, f2=0.00015, f3=-0.10437, f4=0.04869, f5=0.18333, f6=-0.21864, f7=0.27638, f8=-0.13441, label=-115.011801582),
 Row(f1=1.0, f2=-0.00015, f3=0.27118, f4=0.14526, f5=0.06101, f6=0.13401, f7=0.06237, f8=-0.74065, label=-122.623452696),
 Row(f1=1.0, f2=-6e-05, f3=0.1413, f4=0.12084, f5=0.05452, f6=0.09272, f7=0.2534, f8=-0.65331, label=-117.130523174),
 Row(f1=1.0, f2=-8e-05, f3=-0.41534, f4=-0.04205, f5=-0.00724, f6=-0.07463, f7=0.13273, f8=0.19112, label=-73.5775668047),
 Row(f1=1.0, f2=-8e-05, f3=-0.45937, f4=-0.23509, f5=-0.05679, f6=0.06077, f7=-0.49597, f8=-0.30668, label=-137.37933148),
 Row(f1=0.0, f2=2e-05, f3=-0.23465, f4=0.07345, f5=-0.07217, f6=-0.19256, f7=-0.14377, f8=-0.15183, label=-162.804738349),
 Row(f1=0.0, f2=-7e-05, f3=-0.10321, f4=0.27467, f5=0.04058, f6=-0.24541, f7=0.08631, f8=-0.2979, label=-212.111291232),
 Row(f1=1.0, f2=-7e-05, f3=-0.01039, f4=-0.00453, f5=-0.01352, f6=-0.05199, f7=-0.3772, f8=-0.19641, label=-91.5022329392)]
```


In [178]:
#a = requests_rdd.take(1)[0]
def process_line(x):
    ls = []
    x = x.split("|")
    for j in range(len(x)):
        if " => " in x[j]:
            d = x[j].split(" => ")
            ls.append(float(d[0]))
            ls.append(float(d[1]))
        else:
            if x[j] == "SERVER":
                ls.append(0.0)
            elif x[j]=="HOST":
                ls.append(1.0)
            else:
                ls.append(float(x[j]))
    return ls

In [179]:
# try it here
rdd1 = requests_rdd.map(process_line)

In [180]:
# 5 pts
np.testing.assert_equal(len(requests_rdd.map(process_line).first()), 9)
np.testing.assert_equal(requests_rdd.map(process_line).count(), 10000)

## Question 2:

Transform the `requests_rdd` RDD into a Spark 2.0 DataFrame and store it in `requests_df`

In [203]:
requests_df = rdd1.toDF()
requests_df = requests_df.withColumnRenamed("_1","f1").withColumnRenamed("_2","f2").withColumnRenamed("_3","f3").withColumnRenamed("_4","f4").withColumnRenamed("_5","f5").withColumnRenamed("_6","f6").withColumnRenamed("_7","f7").withColumnRenamed("_8","f8").withColumnRenamed("_9","label")
type(requests_df)

pyspark.sql.dataframe.DataFrame

In [182]:
# 5 pts
np.testing.assert_equal(type(requests_df), sql.dataframe.DataFrame)
np.testing.assert_equal(set(requests_df.columns), {'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'label'})
np.testing.assert_equal(requests_df.count(), 10000)

## Question 3:

In this question, we will explore the data. We have a hypothesis that depending on whether the request was from the "HOST" or "SERVER" (`f1` column), there are significant difference in the outcome (`label` column).

You will find whether this is true by computing two quantities for each group of `f1`. You will compute the mean outcome and the *standard error of the mean* or SE of the outcome. The equation for SE of a variable $x$ is:

$$\text{SE}(x) = \frac{\text{std}(x)}{\sqrt{n}}$$

From `requests_df`, create a dataframe `summary_df` that contains, for each value of `f1`, the mean `label` as a column `mlabel` and the SEM of `label` as a column `semlabel`. For the SE equation, use the *sample standard devivation* computed by `fn.stddev_samp`. **Hint: perform an aggregate operation and use appropriate combinations of functions in the package `fn`. Rename columns appropriately**

In [200]:
mlbs = requests_df.groupBy("f1").agg({"label":"mean"})
mlbs = mlbs.withColumnRenamed("avg(label)","mlabel")

stlbs = requests_df.groupBy('f1').agg({"label":"stddev_samp"})
#stlbs.show(2)
nlbs = requests_df.groupBy('f1').agg({"label":"count"})
#nlbs.show(2)
summary_df = stlbs.join(nlbs,'f1')
#summary_df.show(2)
summary_df = summary_df.withColumn("semlabel",stlbs["stddev_samp(label)"]/fn.sqrt(nlbs["count(label)"])) 
summary_df = summary_df.join(mlbs,"f1")
summary_df = summary_df.drop("stddev_samp(label)","count(label)")
summary_df.show(2)

+---+------------------+-------------------+
| f1|          semlabel|             mlabel|
+---+------------------+-------------------+
|0.0|1.7554606758419875|-29.611753412328927|
|1.0|1.7481077347771363| -12.62243193686321|
+---+------------------+-------------------+



The schema of `summary_df` should look like:

```python
summary_df.printSchema()
```
```console
root
 |-- f1: double (nullable = true)
 |-- mlabel: double (nullable = true)
 |-- semlabel: double (nullable = true)

```
The mean label for each `f1` feature should be:

```python
summary_df.select('f1', 'mlabel').show()
```

```console
+---+------------------+
| f1|            mlabel|
+---+------------------+
|0.0|-29.61175341232892|
|1.0|-12.62243193686321|
+---+------------------+
```

In [201]:
# 10 pts
np.testing.assert_equal(summary_df.count(), 2)
np.testing.assert_equal(set(summary_df.columns), {'f1', 'mlabel', 'semlabel'})
np.testing.assert_approx_equal(summary_df.rdd.map(lambda r: r.mlabel).sum(), -42.23418534919213,
                              significant=3)
np.testing.assert_approx_equal(summary_df.rdd.map(lambda r: r.semlabel).sum(), 3.503568410619124,
                              significant=3)

Statistically, we say that a mean $\overline{x}$ is *statistically* different from another mean $\overline{y}$ if it is not within 2 standard errors of the mean of x (i.e., $\overline{y} \not\in \{\overline{x} \pm 2\text{SE}(x)\}$). Discuss whether you think the mean label for f1 = 0 is statistically different from the mean label for f1 = 1

In [204]:
# display it to make your argument
summary_df.show()

+---+------------------+-------------------+
| f1|          semlabel|             mlabel|
+---+------------------+-------------------+
|0.0|1.7554606758419875|-29.611753412328927|
|1.0|1.7481077347771363| -12.62243193686321|
+---+------------------+-------------------+



**3 pts for your answer below:**

### Summary
#### For f1 = 0.0:
$semlabel_0$ = 1.7554606758419875

$mlabel_0$ = -29.611753412328927

#### For f1 = 1.0:
$semlabel_1$ = 1.7481077347771363

$mlabel_1$ = -12.62243193686321

## Question 4:

Use the transformer `VectorAssembler` to create a dataframe that puts all columns `f1`, `f2`, ..., `f8` from `requests_df` into a column named `features`. Assign the vector assembler object into a variable `va` and the new dataframe into the variable  `features_df`

In [222]:
# YOUR CODE HERE
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=["f1", "f2","f3","f4","f5","f6","f7","f8"],outputCol="features")
features_df = va.transform(requests_df)

The schema of the new dataframe should be like this:

```python
features_df.printSchema()
```

```console
root
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- f3: double (nullable = true)
 |-- f4: double (nullable = true)
 |-- f5: double (nullable = true)
 |-- f6: double (nullable = true)
 |-- f7: double (nullable = true)
 |-- f8: double (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
```

In [223]:
# try it here
features_df.show(3)

+---+-------+--------+--------+--------+-------+--------+--------+--------------+--------------------+
| f1|     f2|      f3|      f4|      f5|     f6|      f7|      f8|         label|            features|
+---+-------+--------+--------+--------+-------+--------+--------+--------------+--------------------+
|0.0| 8.0E-5| 0.45592| 0.25362|-0.12878|-0.0693| -0.1474|-0.53736|-106.947151761|[0.0,8.0E-5,0.455...|
|0.0|-1.0E-5| 0.10405|-0.17047| 0.10127|0.13776| 0.66619| 0.32221| 127.061566761|[0.0,-1.0E-5,0.10...|
|1.0|-1.0E-5|-0.72394| 0.31161| 0.06975| 0.0086|-0.44817|-0.22976|-170.222653879|[1.0,-1.0E-5,-0.7...|
+---+-------+--------+--------+--------+-------+--------+--------+--------------+--------------------+
only showing top 3 rows



In [224]:
# 5 pts
np.testing.assert_equal(type(features_df), sql.dataframe.DataFrame)
np.testing.assert_equal(set(features_df.columns), 
                        {'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'features', 'label'})

## Question 5:

Run a linear regression model on `features_df` using the `features` column to predict the `label` column. Store the transformer fit to the data in the `lr_model` variable (the transformer is what the estimator's `fit` function returns). Use the transformer to create a dataframe named `predictions_df` with two columns: `label` and `prediction` based on the `features_df` dataframe.

In [231]:
# YOUR CODE HERE
lr = regression.LinearRegression(featuresCol="features",labelCol="label",maxIter=10, regParam=0.4, elasticNetParam=0.9)
lr_model = lr.fit(features_df)

predictions_df = lr_model.transform(features_df)
predictions_df = predictions_df.select("label","prediction")
predictions_df.show(5)

+--------------+-------------------+
|         label|         prediction|
+--------------+-------------------+
|-106.947151761|  -98.1371488916323|
| 127.061566761|  107.1452180178212|
|-170.222653879|-164.65768779095717|
|-127.850350049|-108.04111399145371|
|-110.295233798|-109.93527884136866|
+--------------+-------------------+
only showing top 5 rows



The resulting dataframe should be in `predictions_df`. Running `predictions_df.show(5)` should produce something like

```python
predictions_df.show(5)
```

```console
+--------------+-------------------+
|         label|         prediction|
+--------------+-------------------+
| 163.877101489| 159.06994708337518|
|-105.023368852| -99.52598722329135|
|-115.011801582|-109.91382979074436|
|-122.623452696|-118.62864861627764|
|-117.130523174|-116.89245751669506|
+--------------+-------------------+
only showing top 5 rows
```


In [232]:
# 5 pts:
np.testing.assert_equal(set(predictions_df.columns), {'label', 'prediction'})
np.testing.assert_equal(predictions_df.count(), 10000)
np.testing.assert_equal(type(lr_model), regression.LinearRegressionModel)
np.testing.assert_equal(type(va), feature.VectorAssembler)

The root mean squared error is defined as

$$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2}$$

Combine functions in `fn` package and other functions to create a dataframe called `rmse_df` that contains the root mean squared error in column `rmse` based on the `predictions_df` dataframe.

In [233]:
# YOUR CODE HERE
rms = predictions_df.withColumn("error",(predictions_df["prediction"]-predictions_df["label"]))
rms = rms.withColumn("squared error",(rms["error"]**2))

rmse_df = rms.groupBy().agg({"squared error":"sum"})
n = rms.groupBy().agg({"error":"count"}).collect()[0][0]

rmse_df = rmse_df.withColumn("rmse", fn.sqrt((1/n*rmse_df["sum(squared error)"])))
rmse_df = rmse_df.drop("sum(squared error)")
rmse_df.show(1)

+-----------------+
|             rmse|
+-----------------+
|8.559748270091829|
+-----------------+



In [234]:
# 5 pts
np.testing.assert_array_less(rmse_df.first().rmse, 10)
np.testing.assert_equal(rmse_df.count(), 1)

## Question 6

Add the constant 10 to each feature in `requests_df` and redo the analytics pipeline from Q5, creating a vector assembler `va2`, `features2_df`, `lr_model2`, and `predictions2_df`. When adding the constant 10, preserve the column names in `features2_df` such that the columns in this new dataframe should be `f1` thorugh `f8`. Do not modify the column `label`. Fit a new linear model, similar to question 4, compute its root mean square error, and store it in `rmse2_df`.

In [246]:
# YOUR CODE HERE
requests_df2 = requests_df.select(requests_df["f1"]+10, requests_df["f2"]+10, requests_df["f3"]+10,requests_df["f4"]+10, requests_df["f5"]+10, requests_df["f6"]+10, requests_df["f7"]+10, requests_df["f8"]+10, requests_df["label"])
cols2 = ["f1","f2","f3","f4","f5","f6","f7","f8","label"]
req2 = requests_df2.toDF(*cols2)

va2 = VectorAssembler(inputCols=["f1", "f2","f3","f4","f5","f6","f7","f8"],outputCol="features")
features_df2 = va.transform(req2)
features_df2.show(3)

lr2 = regression.LinearRegression(featuresCol="features",labelCol="label",maxIter=10, regParam=0.4, elasticNetParam=0.9)
lr_model2 = lr.fit(features_df2)

predictions2_df = lr_model2.transform(features_df2)
predictions2_df = predictions2_df.select("label","prediction")
predictions2_df.show(5)

rms2 = predictions2_df.withColumn("error",(predictions2_df["prediction"]-predictions2_df["label"]))
rms2 = rms2.withColumn("squared error",(rms2["error"]**2))

rmse2_df = rms2.groupBy().agg({"squared error":"sum"})
n2 = rms2.groupBy().agg({"error":"count"}).collect()[0][0]

rmse2_df = rmse2_df.withColumn("rmse", fn.sqrt((1/n*rmse2_df["sum(squared error)"])))
rmse2_df = rmse2_df.drop("sum(squared error)")
rmse2_df.show(1)

+----+--------+--------+--------+--------+--------+--------+--------+--------------+--------------------+
|  f1|      f2|      f3|      f4|      f5|      f6|      f7|      f8|         label|            features|
+----+--------+--------+--------+--------+--------+--------+--------+--------------+--------------------+
|10.0|10.00008|10.45592|10.25362| 9.87122|  9.9307|  9.8526| 9.46264|-106.947151761|[10.0,10.00008,10...|
|10.0| 9.99999|10.10405| 9.82953|10.10127|10.13776|10.66619|10.32221| 127.061566761|[10.0,9.99999,10....|
|11.0| 9.99999| 9.27606|10.31161|10.06975| 10.0086| 9.55183| 9.77024|-170.222653879|[11.0,9.99999,9.2...|
+----+--------+--------+--------+--------+--------+--------+--------+--------------+--------------------+
only showing top 3 rows

+--------------+-------------------+
|         label|         prediction|
+--------------+-------------------+
|-106.947151761| -98.13736777938902|
| 127.061566761|  107.1452447141055|
|-170.222653879|-164.65767319733277|
|-127.85035

In [247]:
# 10 pts:
np.testing.assert_equal(set(predictions2_df.columns), {'label', 'prediction'})
np.testing.assert_equal(predictions2_df.count(), 10000)
np.testing.assert_equal(type(va2), feature.VectorAssembler)
np.testing.assert_equal(type(lr_model2), regression.LinearRegressionModel)
np.testing.assert_array_less(rmse2_df.first().rmse, 10)
np.testing.assert_equal(rmse2_df.count(), 1)

Compare the root mean squared error of Q5 and Q6. They should be almost exactly the same. Also, compare the coefficients and intercepts of `lr_model` and `lr_model2`. 

In [248]:
print("RMSE Q5")
rmse_df.show()
print("RMSE Q6")
rmse2_df.show()
print("intercept lr_model", lr_model.intercept)
print("intercept lr_model2", lr_model2.intercept)
print("coefficients lr_model", lr_model.coefficients)
print("coefficients lr_model2", lr_model2.coefficients)

RMSE Q5
+-----------------+
|             rmse|
+-----------------+
|8.559748270091829|
+-----------------+

RMSE Q6
+-----------------+
|             rmse|
+-----------------+
|8.559762278445925|
+-----------------+

intercept lr_model -28.66290591131293
intercept lr_model2 -1661461.0943377328
coefficients lr_model [17.386615157246727,165401.2576977343,141.53607850862386,0.0,0.0,358.0885257641787,0.0,227.81736205585145]
coefficients lr_model2 [17.386612760053062,165398.41455542966,141.53609398619864,0.0,0.0,358.08852403795674,0.0,227.81735682599646]


**2 pts for your answer below:** Comment on why there are differences or similarities between coefficients, intercepts, RMSE, even though we modified the features. You can use [Latex in Markdown](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html#LaTeX-equations) to put some math into your explanation.

**Coefficients**: Does not change much, because changing the data-points' value by 10 does not change the slope of the linear model.

**Intercepts**: The increase in the value of data-points by 10 has led to an increase in the intercept, which will be the 10*(Coefficients)

$y = b_0 + b_1x_1 + b_2x_2 + b_3x_3$ changes to $y = b_0+b_1(x_1+10)+b_2(x_2+10)+b_3(x_3+10)=(b_0+10b_1+10b_2+10b_3)b_1x_1+b_2x_2+b_3x_3$ which is added to the intercept.

**RMSE**: Does not change much, because deviation from the regression line does not change by increasing the values of $x_1, x_2, and x_3$.